### HSML 6295 Session 4 (Cross-Validation and Model Selection) - ANSWERS

#### I. Cross-Validation

We use a data set that I created using the ad-hoc query system wonder.cdc.gov, which accesses large online databases and is maintained by the Centers for Disease Control and Prevention.

This data set includes crude death rates per 100,000 for ages 0-84 for years 2010-2016.

Read in the data set


In [ ]:
rates = read.csv("HSML 6295 s4 US Death Rates by Age.csv") 
str(rates)
plot(Death_Rate ~ Age, data = rates, 
     xlab = "Age (Years)", ylab = "Deaths per 100,000")
plot(Death_Rate ~ Age, data = rates, 
     xlab = "Age (Years)", ylab = "Deaths per 100,000", xlim = c(0,30), ylim = c(0,1000))
plot(log(Death_Rate) ~ Age, data = rates, 
     xlab = "Age (Years)", ylab = "log(Deaths per 100,000)")
plot(log(Death_Rate) ~ Age, data = rates, 
     xlab = "Age (Years)", ylab = "log(Deaths per 100,000)", 
     xlim = c(0,30), ylim = c(2,7))


In [ ]:
library(boot)

# Set the value of the randomization seed
  set.seed(1)

# Define subset of data frame
  subset = subset(rates, Age >= 0 & Year >= 2016)                   

# Set number of folds
  k=5

# Matrix of MSE results
  (cv.error = rbind(rep(0,10), rep(0,10), rep(0,10)))

# Loop over polynomial degrees
for (i in 1:10){
# Fit polynomial on subset
  glm.fit = glm(Death_Rate ~ poly(Age, i), data = subset)         
# Predict response on full data set
  pred = predict(glm.fit, subset)                                 
# Training MSE
  cv.error[1,i] = round(mean((subset$Death_Rate - pred)^2),0)     
# LOOCV Test MSE
  cv.error[2,i] = round(cv.glm(subset, glm.fit)$delta[1],0)       
# K-Fold CV Test MSE
  cv.error[3,i] = round(cv.glm(subset, glm.fit, K=k)$delta[1],0)  
}

# Extract the values of the polynomial degree that minimize the MSEs
arg_min_train = cv.error[1,][which.min(cv.error[1,])]
arg_min_loocv = cv.error[2,][which.min(cv.error[2,])]
arg_min_cv    = cv.error[3,][which.min(cv.error[3,])]

# Plot the MSE values for polynomial degrees 1 through 10
plot(log(cv.error[1,]), type="l", xlab="Degree of Polynomial", ylab="log(MSE)",
     col="red", lwd=1, pch=15, cex = 0.8, bty="l")
lines(log(cv.error[2,]), type="l", col="blue", lwd=1, pch=16, cex = 0.8)
lines(log(cv.error[3,]), type="l", col="purple", lwd=1, pch=17, cex = 0.8)
points(which.min(cv.error[1,]),log(arg_min_train),col="red",cex=1,pch=15)
points(which.min(cv.error[2,]),log(arg_min_loocv),col="blue",cex=1,pch=16)
points(which.min(cv.error[3,]),log(arg_min_cv),col="purple",cex=1,pch=17)
legend("top", legend =  c(paste("Training MSE:", arg_min_train), 
                          paste("LOOCV Test MSE:", arg_min_loocv), 
                          paste(k,"- Fold CV Test MSE:",arg_min_cv)), 
       col = c("red","blue","purple"), pch = c(15,16,17), bty = "n", 
       text.col = c("red","blue","purple"), horiz = F)


**Knowledge Check 1**

Compute the training MSE, the LOOCV test MSE, and the 5-fold CV test MSE for the age ranges shown in the table below and find the value of the complexity parameter (the degree of the polynomial) that minimizes each error measure. Do the shapes of the graphs representing the 3 measures change when you change the value of the randomization seed?

Hint: Modify the following line in the code chunk above

`subset = subset(rates, Age >= 0 & Year >= 2016)`


| Age Range (Years)                        | 0-84 | 1-84 | 2-84 | 20-84 | 40-84 | 65-84 |
| ---                                      | ---: | ---: | ---: | ---:  | ---:  | ---:  |
| Degree That Minimizes Training MSE       |   10 |      |      |       |       |       |
| Degree That Minimizes LOOCV Test MSE     |    6 |      |      |       |       |       |
| Degree That Minimizes 5-Fold CV Test MSE |    6 |      |      |       |       |       |


**Answer:**

| Age Range (Years)                        | 0-84 | 1-84 | 2-84 | 20-84 | 40-84 | 65-84 |
| ---                                      | ---: | ---: | ---: | ---:  | ---:  | ---:  |
| Degree That Minimizes Training MSE       |   10 |   10 |   10 |     9 |    10 |    10 |
| Degree That Minimizes LOOCV Test MSE     |    6 |   10 |    8 |     9 |     5 |     4 |
| Degree That Minimizes 5-Fold CV Test MSE |    6 |   10 |    8 |     9 |     5 |     4 |


**Knowledge Check 2**

Compute the training MSE, the LOOCV test MSE, and the 5-fold CV test MSE for the age range 0-84 but now change the value of the randomization seed as shown in the table below and find the value of the complexity parameter (the degree of the polynomial) that minimizes each error measure. Do the shapes of the graphs representing the 3 measures change when you change the value of the randomization seed?

Hint: Modify the following line in the code chunk above

`set.seed(1)`



| Seed Value                               |    1 |    2 |    3 |    4 |    6 | 
| ---                                      | ---: | ---: | ---: | ---: | ---: | 
| Degree That Minimizes Training MSE       |   10 |      |      |      |      |
| Degree That Minimizes LOOCV Test MSE     |    6 |      |      |      |      |
| Degree That Minimizes 5-Fold CV Test MSE |    6 |      |      |      |      |

**Answer:**

| Seed Value                               |    1 |    2 |    3 |    4 |    6 | 
| ---                                      | ---: | ---: | ---: | ---: | ---: | 
| Degree That Minimizes Training MSE       |   10 |   10 |   10 |   10 |   10 |
| Degree That Minimizes LOOCV Test MSE     |    6 |    6 |    6 |    6 |    6 |
| Degree That Minimizes 5-Fold CV Test MSE |    6 |   10 |    6 |    6 |   10 |

**Knowledge Check 3**

Compute the K-fold CV test MSE for the age range 0-84 and seed value 1 but now change the number of folds as shown in the table below and find the value of the complexity parameter (the degree of the polynomial) that minimizes the error measure.

Hint: Modify the following line in the code chunk above

`k=5`



| Number of Folds (K)                      |    2 |    3 |    4 |    5 |    6 |    7 |  
| ---                                      | ---: | ---: | ---: | ---: | ---: | ---: | 
| Degree That Minimizes K-Fold CV Test MSE |      |      |      |    6 |      |      |

**Answer:**

| Number of Folds (K)                      |    2 |    3 |    4 |    5 |    6 |    7 |  
| ---                                      | ---: | ---: | ---: | ---: | ---: | ---: | 
| Degree That Minimizes K-Fold CV Test MSE |    7 |    6 |   10 |    6 |    6 |    7 |


**Knowledge Check 4**

Compute the training MSE, the LOOCV test MSE, and the 5-fold CV test MSE for the age range 65-84 and seed value 1 but now change the range of years as shown in the table below and find the value of the complexity parameter (the degree of the polynomial) that minimizes each error measure. Also pay attention to the shape of the graphs representing the 3 measures and in particular to the distance between the value of the training MSE and the two test MSEs when the degree is 10.

Hint: Modify the following line in the code chunk above

`subset = subset(rates, Age >= 0 & Year >= 2016)`


| Years Included                           | 2016 | 2015-2016 | 2013-2016 | 2010-2016 |
| ---                                      | ---: |      ---: |      ---: |     ---:  |
| Degree That Minimizes Training MSE       |   10 |           |           |           |
| Degree That Minimizes LOOCV Test MSE     |    4 |           |           |           |
| Degree That Minimizes 5-Fold CV Test MSE |    4 |           |           |           |

**Answer:**

| Years Included                           | 2016 | 2015-2016 | 2013-2016 | 2010-2016 |
| ---                                      | ---: |      ---: |      ---: |      ---: |
| Degree That Minimizes Training MSE       |   10 |        10 |        10 |        10 |
| Degree That Minimizes LOOCV Test MSE     |    4 |         4 |         4 |         4 |
| Degree That Minimizes 5-Fold CV Test MSE |    4 |         3 |         8 |         3 |

#### II. Summary Statistics for the "Physician Office Visits" Data Set

Load data set


In [ ]:
visits = read.csv("HSML 6295 s4 Visits.csv")
str(visits)



Show summary statistics for continuous variables


In [ ]:
library(stargazer)
stargazer(visits, 
          type = "text", 
          summary.stat = c("n", "mean", "sd", "min", "p25", "median", "p75", "max"),
          title="Descriptive statistics", digits=1)


#### III. Compute the Least Squares Fit (No Subset Selection or Shrinkage)



In [ ]:
summary(lm(Visits ~ ., visits))



**Knowledge Check 5**

Load the "HSML 6295 s4 Contacts.csv" data set and compute summary statistics and the least squares estimates.

**Answer:**


In [ ]:
contacts = read.csv("HSML 6295 s4 Contacts.csv")
str(contacts)
library(stargazer)
stargazer(contacts, 
          type = "text", 
          summary.stat = c("n", "mean", "sd", "min", "p25", "median", "p75", "max"),
          title="Descriptive statistics", digits=1)
summary(lm(Visits ~ ., contacts))


#### IV. Subset Selection

Compute RSS and adjusted training errors for all possible *j*-variable models


In [ ]:
library(leaps)
predictors = ncol(visits)-1
regfit.full = regsubsets(Visits ~ ., data = visits, nvmax=predictors)
(reg.summary = summary(regfit.full))


**IV.A. Indirectly Estimate the Test Error (Adjust the Training Error) **



In [ ]:
# R-Squared
plot(reg.summary$rsq ,xlab="Number of Predictors",ylab="R-Squared",type="l")
points(which.max(reg.summary$rsq),reg.summary$rsq[which.max(reg.summary$rsq)], 
       col="red",cex=2,pch=20)
plot(regfit.full,scale="r2")

# Adjusted R-Squared
plot(reg.summary$adjr2 ,xlab="Number of Predictors",ylab="Adjusted R-Squared",type="l")
points(which.max(reg.summary$adjr2),reg.summary$adjr2[which.max(reg.summary$adjr2)], 
       col="red",cex=2,pch=20)
plot(regfit.full,scale="adjr2")

# C_p
plot(reg.summary$cp ,xlab="Number of Predictors",ylab="Cp", type="l")
points(which.min(reg.summary$cp),reg.summary$cp[which.min(reg.summary$cp)],
       col="red",cex=2,pch=20)
plot(regfit.full,scale="Cp")

# BIC
plot(reg.summary$bic ,xlab="Number of Predictors",ylab="BIC", type="l")
points(which.min(reg.summary$bic),reg.summary$bic[which.min(reg.summary$bic)],
       col="red",cex=2,pch=20)
plot(regfit.full,scale="bic")


**Knowledge Check 6**

Find the optimal number of predictors based on the adjusted training errors Adjusted $R^2$, $C_p$, and $BIC$.

**Answer:**


In [ ]:
library(leaps)
predictors = ncol(contacts)-1
regfit.full = regsubsets(Visits ~ ., data = contacts, nvmax=predictors)
reg.summary = summary(regfit.full)

# Adjusted R-Squared
plot(reg.summary$adjr2 ,xlab="Number of Predictors",ylab="Adjusted R-Squared",type="l")
points(which.max(reg.summary$adjr2),reg.summary$adjr2[which.max(reg.summary$adjr2)], 
       col="red",cex=2,pch=20)

# C_p
plot(reg.summary$cp ,xlab="Number of Predictors",ylab="Cp", type="l")
points(which.min(reg.summary$cp),reg.summary$cp[which.min(reg.summary$cp)],
       col="red",cex=2,pch=20)

# BIC
plot(reg.summary$bic ,xlab="Number of Predictors",ylab="BIC", type="l")
points(which.min(reg.summary$bic),reg.summary$bic[which.min(reg.summary$bic)],
       col="red",cex=2,pch=20)


**For the following three performance metrics, the optimal number of predictors is**

**Adjusted $R^2$** 


In [ ]:
which.max(reg.summary$adjr2)




**$C_p$:**


In [ ]:
which.min(reg.summary$cp)




**$BIC$:**


In [ ]:
which.min(reg.summary$bic)



**IV.B. Directly Estimate the Test Error (Cross-Validation)**

Define the `predict` function


In [ ]:
predict.regsubsets = function (object, newdata, id,...){
  form=as.formula(object$call[[2]])
  mat=model.matrix(form,newdata)
  coefi=coef(object, id=id)
  xvars=names(coefi)
  mat[,xvars]%*%coefi
}



Define the model tuning parameter values


In [ ]:
k=10
set.seed (1)
folds=sample(1:k,nrow(visits),replace=TRUE)
predictors = ncol(visits)-1
cv.errors=matrix(NA, k, predictors, dimnames=list(NULL, paste(1:predictors)))



1. Compute $k$ (cross-validated) test errors for all $j$-variable models


In [ ]:
for(j in 1:k){
  best.fit = regsubsets(Visits ~ ., data = visits[folds!=j,], nvmax = predictors)
  for(i in 1:predictors){
    pred = predict(best.fit, visits[folds==j,], id=i)
    cv.errors[j,i] = mean((visits$Visits[folds==j]-pred)^2)
  }
}
cv.errors



2. Average over the $k$ CV test errors and plot the means


In [ ]:
(mean.cv.errors=apply(cv.errors ,2 ,mean))
arg_min_cv = which.min(mean.cv.errors)
min_cv     = round(mean.cv.errors[arg_min_cv],2)
  
par(mfrow=c(1,1))
labels = matrix(c("Minimum 10-Fold CV Test MSE:", min_cv), nrow=1, ncol=2) 
mylabels = paste(labels[,1], labels[,2])
plot(mean.cv.errors, type="l", xlab="Number of Predictors", ylab="Average CV Test Error",
     col="darkblue", lwd=1, pch=15, cex = 0.8, bty="l")
points(arg_min_cv,min_cv,col="darkblue",cex=1,pch=16)
legend("top", legend = mylabels, col = c("darkblue"), pch = c(16), bty = "n", 
       text.col = c("darkblue"), horiz = F)



3. Reestimate all models with


In [ ]:
arg_min_cv



variables on the *full* data set and retain the one with the smallest RSS


In [ ]:
reg.best = regsubsets(Visits ~ ., data = visits, nvmax = predictors)
(summary(reg.best))                   # Show which predictors to retain
round(coef(reg.best,arg_min_cv),2)    # Show coefficient estimates
round(lm(Visits ~ ., visits)$coef,2)  # Compare to full model ("the least squares fit")


#### V. Shrinkage Methods

**V.A. Setup**

Declare matrix of predictors x and response variable y


In [ ]:
x = model.matrix(Visits ~ ., data = visits)[,-1] 
y = visits$Visits



Split data set into training and test sets


In [ ]:
set.seed (1)
train = sample(1:nrow(x), round(nrow(x)/2))
test = (-train)
y.test = y[test]



Load `glmnet` package


In [ ]:
library(glmnet)



**V.B. Ridge Regression**

Compute value of $\lambda$ (lambda) that minimizes the CV MSE for the *training* set


In [ ]:
set.seed (1)
cv.out = cv.glmnet(x[train,], y[train], alpha=0)
plot(cv.out)
(bestlam = cv.out$lambda.min)
log(bestlam)



Compute the test MSE


In [ ]:
ridge.mod=glmnet(x[train,],y[train],alpha=0,lambda=bestlam)
ridge.pred=predict(ridge.mod,s=bestlam ,newx=x[test,])
mean((ridge.pred-y.test)^2)



Print coefficient estimates based on optimal lambda $\lambda^{\ast}$ and full data set


In [ ]:
out = glmnet(x,y,alpha=0,lambda=bestlam)
round(predict(out, type="coefficients", s = bestlam)[1:nrow(coef(out)),],2)
round(lm(Visits ~ ., visits)$coef,2)  # Compare to full model ("the least squares fit")


**V.C. The Lasso**

Compute value of $\lambda$ (lambda) that minimizes the CV MSE for the *training* set


In [ ]:
set.seed (1)
cv.out = cv.glmnet(x[train,], y[train], alpha=1)
plot(cv.out)
(bestlam = cv.out$lambda.min)
log(bestlam)



Compute the test MSE


In [ ]:
ridge.mod=glmnet(x[train,],y[train],alpha=1,lambda=bestlam)
ridge.pred=predict(ridge.mod,s=bestlam ,newx=x[test,])
mean((ridge.pred-y.test)^2)



Print coefficient estimates based on optimal lambda $\lambda^{\ast}$ and full data set


In [ ]:
out = glmnet(x,y,alpha=1,lambda=bestlam)
round(predict(out, type="coefficients", s = bestlam)[1:nrow(coef(out)),],2)
round(lm(Visits ~ ., visits)$coef,2)  # Compare to full model ("the least squares fit")


**Knowledge Check 7**

Find the value of $\lambda$ and the coefficient estimates of the best model based on ridge regression.

**Answer:**


In [ ]:
x = model.matrix(Visits ~ ., data = contacts)[,-1] 
y = contacts$Visits

set.seed (1)
train = sample(1:nrow(x), round(nrow(x)/2))
test = (-train)
y.test = y[test]

library(glmnet)

set.seed (1)
cv.out = cv.glmnet(x[train,], y[train], alpha=0)
plot(cv.out)
(bestlam = cv.out$lambda.min)
log(bestlam)

out = glmnet(x,y,alpha=0)
round(predict(out, type="coefficients", s = bestlam)[1:nrow(coef(out)),],2)
round(lm(Visits ~ ., contacts)$coef,2)  # Compare to full model ("the least squares fit")


**Knowledge Check 8**

Find the value of $\lambda$ and the coefficient estimates of the best model based on the lasso.

**Answer:**


In [ ]:
x = model.matrix(Visits ~ ., data = contacts)[,-1] 
y = contacts$Visits

set.seed (1)
train = sample(1:nrow(x), round(nrow(x)/2))
test = (-train)
y.test = y[test]

library(glmnet)

set.seed (1)
cv.out = cv.glmnet(x[train,], y[train], alpha=1)
plot(cv.out)
(bestlam = cv.out$lambda.min)
log(bestlam)

out = glmnet(x,y,alpha=1)
round(predict(out, type="coefficients", s = bestlam)[1:nrow(coef(out)),],2)
round(lm(Visits ~ ., contacts)$coef,2)  # Compare to full model ("the least squares fit")
